#PyPDFLoader:
Extracts text from PDF page-by-page.

###Performance Metrics (Loader):

1. Total characters, alphanumeric ratio, and token count indicate extraction volume and noise.
2. Structural preservation uses newline patterns.
3. Processing time and memory usage help evaluate efficiency.

#SemanticChunker:
Uses sentence-level splitting and semantic proximity to generate coherent chunks.

###Performance Metrics (Splitter):

1. Chunk size variability (CV), semantic continuity (via CrossEncoder), and metadata correctness.

2. Measured time and memory usage.

In [1]:
# Install LangChain components and transformers for tokenization and semantic evaluation
!pip install langchain langchain-community langchain-text-splitters
!pip install sentence-transformers transformers psutil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

In [8]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 16.7 MB/s eta 0:00:00


In [2]:
import os
import time
import numpy as np
import psutil

from langchain_community.document_loaders import PyPDFLoader
from transformers import AutoTokenizer

In [3]:
# Load tokenizer to estimate token count (simulating GPT-4 encoding behavior)
tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Replace with GPT-4 tokenizer if needed

# Path to your PDF file
pdf_path = "/content/SystemDesign.pdf"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
# Initialize the loader
loader = PyPDFLoader(pdf_path)

In [5]:
# Start timing and memory tracking
start_time = time.time()
process = psutil.Process(os.getpid())
start_mem = process.memory_info().rss

In [9]:
# Load documents using PyPDFLoader
docs = loader.load()

# Concatenate full text from all pages
full_text = " ".join([doc.page_content for doc in docs])

In [10]:
# ------ Loader Performance Metrics ------
total_chars = len(full_text)
alphanumeric_chars = len([c for c in full_text if c.isalnum()])
newline_chars = full_text.count('\n')
token_count = len(tokenizer.encode(full_text))
content_to_noise_ratio = alphanumeric_chars / total_chars
processing_time_loader = time.time() - start_time
mem_used_loader = (psutil.Process().memory_info().rss - start_mem) / (1024 ** 2)
structural_preservation_score = newline_chars / len(docs)

Token indices sequence length is longer than the specified maximum sequence length for this model (64843 > 1024). Running this sequence through the model will result in indexing errors


In [11]:
loader_metrics = {
    "Total Characters": total_chars,
    "Alphanumeric Characters": alphanumeric_chars,
    "Newline Characters": newline_chars,
    "Token Count": token_count,
    "Content-to-Noise Ratio": round(content_to_noise_ratio, 3),
    "Structural Preservation": round(structural_preservation_score, 3),
    "Processing Time (s)": round(processing_time_loader, 3),
    "Memory Usage (MB)": round(mem_used_loader, 3),
}

print("📊 Loader Metrics:")
for k, v in loader_metrics.items():
    print(f"{k}: {v}")

📊 Loader Metrics:
Total Characters: 283172
Alphanumeric Characters: 229288
Newline Characters: 3552
Token Count: 64843
Content-to-Noise Ratio: 0.81
Structural Preservation: 34.485
Processing Time (s): 38.127
Memory Usage (MB): 67.391


In [13]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 10.6 MB/s eta 0:00:00


In [14]:
from langchain_experimental.text_splitter import SemanticChunker
from sentence_transformers import CrossEncoder

In [15]:
# Initialize semantic coherence model
semantic_model = CrossEncoder("cross-encoder/stsb-roberta-base")

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

In [19]:
# Start splitter timer
start_time_splitter = time.time()

# Initialize embeddings (assuming you have sentence-transformers installed)
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2") # Or any other sentence-transformers model

# Semantic chunking
splitter = SemanticChunker(embeddings=embeddings) # Pass the embeddings object to SemanticChunker
chunks = splitter.split_documents(docs)

splitter_time = time.time() - start_time_splitter

<ipython-input-19-90b8c9b28c9e>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2") # Or any other sentence-transformers model


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
splitter_mem = (psutil.Process().memory_info().rss - start_mem) / (1024 ** 2)

# Get chunk texts
chunk_texts = [chunk.page_content for chunk in chunks]

In [21]:
# Semantic Coherence Testing
semantic_scores = []
for i in range(len(chunk_texts) - 1):
    score = semantic_model.predict([(chunk_texts[i], chunk_texts[i + 1])])
    semantic_scores.append(score)

semantic_flow = float(np.mean(semantic_scores))

In [22]:
# Chunk Size Coefficient of Variation (CV)
chunk_lengths = [len(c) for c in chunk_texts]
chunk_size_cv = np.std(chunk_lengths) / np.mean(chunk_lengths)

In [23]:
# Metadata accuracy estimation
metadata_accuracy = 1.0 if all(hasattr(chunk, "metadata") for chunk in chunks) else 0.0

In [24]:
splitter_metrics = {
    "Chunk Size CV": round(chunk_size_cv, 3),
    "Semantic Flow": round(semantic_flow, 3),
    "Metadata Accuracy": metadata_accuracy,
    "Splitter Processing Time (s)": round(splitter_time, 3),
    "Memory Usage (MB)": round(splitter_mem, 3),
}
print("\n📊 Splitter Metrics:")
for k, v in splitter_metrics.items():
    print(f"{k}: {v}")


📊 Splitter Metrics:
Chunk Size CV: 0.759
Semantic Flow: 0.522
Metadata Accuracy: 1.0
Splitter Processing Time (s): 1356.532
Memory Usage (MB): 2385.102
